In [ ]:
#import the necessary modules 
%matplotlib inline 
import numpy as np 
import matplotlib.pylab as plt 
import pandas as pd 
import scipy
import sklearn
import itertools 
from itertools import cycle 
import os.path as op
import timeit 
import json
import math

# These imports are for creating the percent occupancy heat map 
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
import multiprocessing as m_proc
m_proc.cpu_count()

In [ ]:
# Import MDAnalysis
import MDAnalysis as mda
import statsmodels as stats
from MDAnalysis.analysis import polymer, distances, rdf

### Calculate radius of gyration and persistence length of PLGA/water simulations

In [ ]:
def seg_org(poly_atoms, n_monomers):
    
    pmecl = []
    count = 0
    for i in range(n_monomers):
        fgrp = []
        count += 1
        n_atm = len(poly_atoms.select_atoms("resid "+str(count)))
        
        if count == 1:
            #print(poly_atoms.select_atoms("resid "+str(count)+" and name O13"))
            pmecl.append(poly_atoms.select_atoms("resid "+str(count)+" and (name O13 O9)"))
            #print(pmecl)
        elif count == 2:
            #print(poly_atoms.select_atoms("name O5 and resid "+str(count)))
            pmecl.append(poly_atoms.select_atoms("(resid "+str(count)+" and name O5) or (resid "+str(count-1)+" and name O9)"))
        
        if count != 2 and count != 1:
            if count == n_monomers:
                pmecl.append(poly_atoms.select_atoms("(resid "+str(count)+" and name O1) or (resid "+str(count-1)+" and name O5)"))
            elif count != n_monomers:
            #print(count)
                pmecl.append(poly_atoms.select_atoms("(resid "+str(count)+" and name O5) or (resid "+str(count-1)+" and name O5)"))

    return pmecl

In [ ]:
# write function to average across frames to give ensembled averaged correlation values at each nl value
# write function to average across frames to give ensembled averaged correlation values at each nl value

def pers_length(polymer_atoms, n_monomers):
    
    vec_poly = np.zeros(shape=(3,n_monomers-1), dtype=float)
    
    len_vec = np.arange(n_monomers-1)
    
    len_vec += 1

    vec_poly[1,:] = len_vec

    count = 0 
    
    sv_ply = np.zeros(shape=(n_monomers-1), dtype=object)
    
    for i in range(n_monomers):
        
        count += 1
        
        ds_cor = np.zeros(shape=(n_monomers-count))
        
        for j in range(n_monomers - count):
        
            jh = np.arange(n_monomers - count)
            
            jh += count+1

            n6_mon1 = polymer_atoms.select_atoms("resid "+str(count))

            n6_mon2 = polymer_atoms.select_atoms("resid "+str(jh[j]))

            if j == 0: 
                    
                v1 = n6_mon1.center_of_mass() - n6_mon2.center_of_mass()
                    
                v1_norm = v1/(np.linalg.norm(v1))
            
                ds_cor[j] = v1_norm.dot(v1_norm)

            elif j != 0:
                    
                v2 = n6_mon1.center_of_mass() - n6_mon2.center_of_mass()
                    
                v2_norm = v2/(np.linalg.norm(v2))
                
                ds_cor[j] = np.dot(v1_norm, v2_norm)

            sv_ply[i] = ds_cor
    
    cor_avg = []
    
    for j in range(n_monomers-1):
        lss = []
        for i in sv_ply.flat:
            try:
                lss.append(i[j])
            except IndexError:
                pass
        cor_avg.append(np.mean(lss))
    
    nm = np.array(cor_avg)
    
    ang_vg = []
    for i in nm.flat:
        if i >= float(0.99):
            ang_vg.append(0)
        elif i <= float(0.99):
            ang_vg.append(math.degrees(math.acos(i)))

    vec_poly[0,:] = nm 
    vec_poly[2,:] = np.array(ang_vg)
    
    return vec_poly

In [ ]:
def get_rg_pers_poly(polymer_atoms, universe, start, end, ln_bnd):
    """This function will calculate the average radius of gyration and persistence length of a polymer within a 
    trajectory block."""
    
    n_monomers = len(np.unique(polymer_atoms.resids))
    
    rg_ens = np.zeros(shape=(1,end-start))
    
    corr_v = np.zeros(shape=(n_monomers-1,end-start))
    
    angle_v = np.zeros(shape=(n_monomers-1,end-start))
    
    v_poly = np.zeros(shape=(3,n_monomers-1))
    
    count_rg = 0
    
    universe.trajectory[start]
    
    for ts in universe.trajectory[start:end]:
        
        p_mat = pers_length(polymer_atoms, n_monomers)
     
        corr_v[:,count_rg] = p_mat[0,:]
        
        angle_v[:,count_rg] = p_mat[2,:]
        
        rg_ens[0,count_rg] = polymer_atoms.radius_of_gyration()
        
        count_rg += 1 
        
        universe.trajectory[end]
    
    v_poly[2,:] = p_mat[1,:]
    
    for i in range(n_monomers-1):
        
        v_poly[0,i] = np.mean(corr_v[i,:])
        
        v_poly[1,i] = np.mean(angle_v[i,:])
     
    avg_rg = np.mean(rg_ens)
    
    def expfunc(x, a):
        return np.exp(-x*ln_bnd/a)
    
    tr_n6 = np.arange(n_monomers-1)
    
    #tr_n6 += 1

    pers_lp = scipy.optimize.curve_fit(expfunc, tr_n6, v_poly[0,:])[0][0]
    
    return  rg_ens, v_poly, pers_lp, avg_rg

In [ ]:
# I want a list of total fraction of contacts where length is determined by no. of blocks and a dictionary 
# of contact groups as keys and list of fractional contacts as values(length of list will be no. of blocks)
def bavg_pers_cnt(no_of_blks, polymer_atoms, universe, len_bnd, begin, final):
 
    n_size = (final - begin)/no_of_blks
    
    ot_dab = {}
    universe.trajectory[begin]
    
    sf_lbl = ["Avg Radius of gyration", "Avg persistence length"]

    blk_nparr = np.zeros(shape=(len(sf_lbl),no_of_blks))
    
    count = 0 
    
    for i in range(no_of_blks):
        
        start = universe.trajectory.frame
        print(start)
    
        end = int(start + n_size)
        print(end)
       
        pp_rgens, cor_pp, per_pp, rg_avgpp = get_rg_pers_poly(polymer_atoms, universe, start, end, len_bnd)
        
        blk_nparr[0,count] = rg_avgpp
        
        blk_nparr[1,count] = per_pp        
        
        universe.trajectory[end]
        
        count += 1
    # Save fractional contacts for each AA group type, each element in the value array corresponds to a block 
    # calculated value
    for i in range(len(sf_lbl)):
        ot_dab[sf_lbl[i]] = blk_nparr[i,:]
    
    return ot_dab
        

### Water only systems

In [ ]:
# For the right Rg calculation using MD Analysis, use trajactory without pbc 
n6_plga_wat = mda.Universe("n6_plga_50ns/0_xa_soln/confout.gro", "n6_plga_50ns/0_xa_soln/nopbc_n6wat.xtc")

In [ ]:
len(n6_plga_wat.trajectory)

In [ ]:
#Select the polymer heavy atoms 
poly_n6 = n6_plga_wat.select_atoms("resname sPLG PLG tPLG and not type H")

In [ ]:
o9_atom = poly_n6.select_atoms("resid 1 and name O9")
o5_plg1 = poly_n6.select_atoms("resid 2 and name O5")
lbond = np.zeros(shape=(1,5000))
count = 0
for ts in n6_plga_wat.trajectory[0:5000]:
    oo_len = mda.analysis.distances.distance_array(o9_atom.positions, o5_plg1.positions, 
                                                   box=n6_plga_wat.trajectory.ts.dimensions)
    lbond[0, count] = oo_len
    count += 1

#lbond
#lb_avg = np.mean(lbond)
plt.scatter(np.arange(5000),lbond)

In [ ]:
list(poly_n6)

In [ ]:
poly_n6.center_of_mass()

In [ ]:
n6_plga_wat.select_atoms("resid 1").center_of_mass()

In [ ]:
list(n6_plga_wat.select_atoms("resid 1"))

In [ ]:
n_monomers = len(np.unique(poly_n6.resids))
com_pos_n6 = np.zeros(shape=(n_monomers), dtype=object)
for i in range(n_monomers):
    #print(i)
    #print(n6_plga_wat.select_atoms("resid "+str(i+1)).center_of_mass())
    com_pos_n6[i] = n6_plga_wat.select_atoms("resid "+str(i+1)).center_of_mass()
com_pos_n6

In [ ]:
com_bond = np.zeros(shape=(1,5000))
count = 0
for ts in n6_plga_wat.trajectory[0:5000]:
    n6_mon1 = n6_plga_wat.select_atoms("resid 1")
    n6_mon2 = n6_plga_wat.select_atoms("resid 2")
    oo_len = mda.analysis.distances.distance_array(n6_mon1.center_of_mass(), n6_mon2.center_of_mass(), 
                                                   box=n6_plga_wat.trajectory.ts.dimensions)
    com_bond[0, count] = oo_len
    count += 1
    

In [ ]:
com_bond

In [ ]:
lb_avg = np.mean(com_bond)
lb_avg

In [ ]:
np.std(com_bond)

In [ ]:
np.mean(lbond)

In [ ]:
np.std(lbond)

In [ ]:
plt.scatter(np.arange(5000),com_bond)
plt.scatter(np.arange(5000),lbond,c="y")

In [ ]:
n6_rgens, cor_n6_mat, n6_pers, avg_rg = get_rg_pers_poly(poly_n6, n6_plga_wat, 0, 1000, lb_avg)

In [ ]:
n6_rgens

In [ ]:
cor_n6_mat

In [ ]:
cor_n6_mat[1]

In [ ]:
n6_pers

In [ ]:
avg_rg

In [ ]:
lb_avg

In [ ]:
def exfun(x, a):
    return np.exp(-x*lb_avg/a)
nt = np.arange(5)
#nt += 1
plt.scatter(nt, cor_n6_mat[0,:])
# a-axis is the number of bond lengths away 
plt.plot(nt,exfun(nt, n6_pers))

In [ ]:
n6_blks = bavg_pers_cnt(5, poly_n6, n6_plga_wat, lb_avg, 0 , 5000)

In [ ]:
# Fully extended length of 6 monomer PLGA is 3.838 nm 
n6_blks["Avg persistence length"]

In [ ]:
n6_blks["Avg Radius of gyration"]

In [ ]:
np.mean(n6_blks["Avg persistence length"])

In [ ]:
np.std(n6_blks["Avg persistence length"])

In [ ]:
n6_rgens, cor_n6, pers_n6, rg_avgn6 = get_rg_pers_poly(poly_n6, n6_plga_wat, 2000, 3000, lb_avg)

In [ ]:
cor_n6

In [ ]:
pers_n6

In [ ]:
rg_n6 = np.zeros(shape=(5000))

count = 0
for ts in n6_plga_wat.trajectory[0:5000]:

    rg_n6[count] = poly_n6.radius_of_gyration()
    
    count += 1
    
tr_n6 = np.arange(5000)   
plt.figure(figsize=(8,8))
plt.scatter(tr_n6,rg_n6/10)

In [ ]:
n8_plga_wat = mda.Universe("n8_plga_50ns/0_xa_soln/confout.gro", "n8_plga_50ns/0_xa_soln/nopbc_traj.xtc")

In [ ]:
n8_plga_wat

In [ ]:
#Select the polymer heavy atoms 
poly_n8 = n8_plga_wat.select_atoms("resname sPLG PLG tPLG and not type H")

In [ ]:
len(n8_plga_wat.trajectory)

In [ ]:
list(o5_plg1)

In [ ]:
o9_atom_n8 = poly_n8.select_atoms("resid 1 and name O9")
o5_plg1_n8 = poly_n8.select_atoms("resid 2 and name O5")
lbond = np.zeros(shape=(1,5000))
count = 0
for ts in n8_plga_wat.trajectory[0:5000]:
    oo_len = mda.analysis.distances.distance_array(o9_atom_n8.positions, o5_plg1_n8.positions, 
                                                   box=n8_plga_wat.trajectory.ts.dimensions)
    lbond[0, count] = oo_len
    count += 1

#lbond
lb_avg_n8 = np.mean(lbond)
lb_avg_n8

In [ ]:
n8_blks = bavg_pers_cnt(5, poly_n8, n8_plga_wat, lb_avg, 0 , 5000)

In [ ]:
n8_blks["Avg persistence length"]

In [ ]:
n8_blks["Avg Radius of gyration"]

In [ ]:
n8_rgens, cor_n8, pers_n8, rg_avgn8 = get_rg_pers_poly(poly_n8, n8_plga_wat, 3000, 4000, lb_avg)

In [ ]:
cor_n8

In [ ]:
pers_n8

In [ ]:
def exfun(x, a):
    return np.exp(-x*lb_avg_n8/a)

nt = np.arange(8)
plt.scatter(nt, cor_n8[0,:])
plt.plot(nt,exfun(nt, pers_n8))

In [ ]:
np.mean(n8_blks["Avg persistence length"])

In [ ]:
np.std(n8_blks["Avg persistence length"])

In [ ]:
n10_plga_wat = mda.Universe("n10_plga_50ns/0_xa_soln/N10_sys.gro", "n10_plga_50ns/0_xa_soln/nopbc_10wat.xtc")

In [ ]:
n10_plga_wat

In [ ]:
len(n10_plga_wat.trajectory)

In [ ]:
poly_n10 = n10_plga_wat.select_atoms("resname sPLG PLG tPLG and not type H")

In [ ]:
o9_atom_10 = poly_n10.select_atoms("resid 1 and name O9")
o5_plg_10 = poly_n10.select_atoms("resid 2 and name O5")
lbond = np.zeros(shape=(1,5000))
count = 0
for ts in n10_plga_wat.trajectory[0:5000]:
    oo_len = mda.analysis.distances.distance_array(o9_atom_10.positions, o5_plg_10.positions,
                                                   box=n10_plga_wat.trajectory.ts.dimensions)
    lbond[0, count] = oo_len
    count += 1

#lbond
lb_avg_n10 = np.mean(lbond)
lb_avg_n10

In [ ]:
n10_blks = bavg_pers_cnt(5, poly_n10, n10_plga_wat, lb_avg_n10, 0 , 5000)

In [ ]:
# Extended Length of 10 monomer length plga: 6.436 nm 
n10_blks["Avg persistence length"]

In [ ]:
n10_blks["Avg Radius of gyration"]

In [ ]:
rg_n10 = np.zeros(shape=(5000))

count = 0
for ts in n10_plga_wat.trajectory[0:5000]:

    rg_n10[count] = poly_n10.radius_of_gyration()
    
    count += 1
    
tr_n6 = np.arange(5000)   
plt.figure(figsize=(8,8))
plt.scatter(tr_n6,rg_n10/10)

In [ ]:
np.mean(rg_n10)

In [ ]:
np.mean(n10_blks["Avg persistence length"])

In [ ]:
np.std(n10_blks["Avg persistence length"])

In [ ]:
n20_plga_wat = mda.Universe("n20_plga_150ns/0_xa_soln/nowat_n20.gro", "n20_plga_150ns/0_xa_soln/N20_nopbcpp.xtc")

In [ ]:
n20_plga_wat

In [ ]:
len(n20_plga_wat.trajectory)

In [ ]:
poly_n20 = n20_plga_wat.select_atoms("resname sPLG PLG tPLG and not type H")

In [ ]:
o9_atom_20 = poly_n20.select_atoms("resid 1 and name O9")
o5_plg_20 = poly_n20.select_atoms("resid 2 and name O5")
lbond = np.zeros(shape=(1,15000))
count = 0
for ts in n20_plga_wat.trajectory[0:15000]:
    oo_len = mda.analysis.distances.distance_array(o9_atom_20.positions, o5_plg_20.positions,
                                                   box=n20_plga_wat.trajectory.ts.dimensions)
    lbond[0, count] = oo_len
    count += 1

#lbond
lb_avg_n20 = np.mean(lbond)
lb_avg_n20

In [ ]:
n20_blks = bavg_pers_cnt(5, poly_n20, n20_plga_wat, lb_avg_n20, 0 , 15000)

In [ ]:
n20_blks["Avg persistence length"]

In [ ]:
n20_blks["Avg Radius of gyration"]

In [ ]:
np.mean(n20_blks["Avg persistence length"])

In [ ]:
np.std(n20_blks["Avg persistence length"])

In [ ]:
n30_plga_wat = mda.Universe("n30_plga_150ns/0_xa_soln/0xa_n30sys.gro", "n30_plga_150ns/0_xa_soln/N30_nopbcpp_0xa.xtc")

In [ ]:
n30_plga_wat

In [ ]:
len(n30_plga_wat.trajectory)

In [ ]:
poly_n30 = n30_plga_wat.select_atoms("resname sPLG PLG tPLG and not type H")

In [ ]:
o9_atom_30 = poly_n30.select_atoms("resid 1 and name O9")
o5_plg_30 = poly_n30.select_atoms("resid 2 and name O5")
lbond = np.zeros(shape=(1,15000))
count = 0
for ts in n30_plga_wat.trajectory[0:15000]:
    oo_len = mda.analysis.distances.distance_array(o9_atom_30.positions, o5_plg_30.positions,
                                                   box=n30_plga_wat.trajectory.ts.dimensions)
    lbond[0, count] = oo_len
    count += 1

#lbond
lb_avg_n30 = np.mean(lbond)
lb_avg_n30

In [ ]:
n30_blks = bavg_pers_cnt(5, poly_n30, n30_plga_wat, lb_avg_n30, 0 , 15000)

In [ ]:
n30_blks["Avg persistence length"]

In [ ]:
n30_blks["Avg Radius of gyration"]

In [ ]:
np.mean(n30_blks["Avg persistence length"])

In [ ]:
np.std(n30_blks["Avg persistence length"])

### Acetone systems only

In [ ]:
n6_plga_ace = mda.Universe("n6_plga_50ns/1_xa_soln/confout.gro", "n6_plga_50ns/1_xa_soln/nopbc_n6ace.xtc")

In [ ]:
n6_plga_ace

In [ ]:
len(n6_plga_ace.trajectory)

In [ ]:
poly_n6_ace = n6_plga_ace.select_atoms("resname sPLG PLG tPLG and not type H")

In [ ]:
o9_atom_n6ac = poly_n6_ace.select_atoms("resid 1 and name O9")
o5_plg_n6ac = poly_n6_ace.select_atoms("resid 2 and name O5")
lbond = np.zeros(shape=(1,5000))
count = 0
for ts in n6_plga_ace.trajectory[0:5000]:
    oo_len = mda.analysis.distances.distance_array(o9_atom_n6ac.positions, o5_plg_n6ac.positions,
                                                   box=n6_plga_ace.trajectory.ts.dimensions)
    lbond[0, count] = oo_len
    count += 1

#lbond
lb_avg_n6ace = np.mean(lbond)
lb_avg_n6ace

In [ ]:
lb_avg

In [ ]:
n6_blks_ace = bavg_pers_cnt(5, poly_n6_ace, n6_plga_ace, lb_avg, 0 , 5000)

In [ ]:
n6_blks_ace["Avg persistence length"]

In [ ]:
n6_blks_ace["Avg Radius of gyration"]

In [ ]:
np.mean(n6_blks_ace["Avg persistence length"])

In [ ]:
np.std(n6_blks_ace["Avg persistence length"])

In [ ]:
n8_plga_ace = mda.Universe("n8_plga_50ns/1_xa_soln/confout.gro", "n8_plga_50ns/1_xa_soln/nopbc_n8ace.xtc")

In [ ]:
n8_plga_ace

In [ ]:
len(n8_plga_ace.trajectory)

In [ ]:
poly_n8_ace = n8_plga_ace.select_atoms("resname sPLG PLG tPLG and not type H")

In [ ]:
n8_blks_ace = bavg_pers_cnt(5, poly_n8_ace, n8_plga_ace, lb_avg, 0 , 5000)

In [ ]:
n8_blks_ace["Avg persistence length"]

In [ ]:
n8_blks_ace["Avg Radius of gyration"]

In [ ]:
np.mean(n8_blks_ace["Avg persistence length"])

In [ ]:
np.std(n8_blks_ace["Avg persistence length"])

In [ ]:
n10_plga_ace = mda.Universe("n10_plga_50ns/1_xa_soln/confout.gro", "n10_plga_50ns/1_xa_soln/nopbc_n10ace.xtc")

In [ ]:
n10_plga_ace

In [ ]:
len(n10_plga_ace.trajectory)

In [ ]:
poly_n10_ace = n10_plga_ace.select_atoms("resname sPLG PLG tPLG and not type H")

In [ ]:
n10_blks_ace = bavg_pers_cnt(5, poly_n10_ace, n10_plga_ace, lb_avg, 0 , 5000)

In [ ]:
n10_blks_ace["Avg persistence length"]

In [ ]:
n10_blks_ace["Avg Radius of gyration"]

In [ ]:
np.mean(n10_blks_ace["Avg persistence length"])

In [ ]:
np.std(n10_blks_ace["Avg persistence length"])

In [ ]:
n20_plga_ace = mda.Universe("n20_plga_150ns/1_xa_soln/1xa_n20sys.gro", "n20_plga_150ns/1_xa_soln/N20_nopbcpp_0xa.xtc")

In [ ]:
n20_plga_ace

In [ ]:
len(n20_plga_ace.trajectory)

In [ ]:
poly_n20_ace = n20_plga_ace.select_atoms("resname sPLG PLG tPLG and not type H")

In [ ]:
n20_blks_ace = bavg_pers_cnt(5, poly_n20_ace, n20_plga_ace, lb_avg, 0 , 15000)

In [ ]:
n20_blks_ace["Avg persistence length"]

In [ ]:
n20_blks_ace["Avg Radius of gyration"]

In [ ]:
np.mean(n20_blks_ace["Avg persistence length"])

In [ ]:
np.std(n20_blks_ace["Avg persistence length"])

In [ ]:
n30_plga_ace = mda.Universe("n30_plga_150ns/1_xa_soln/1xa_n30sys.gro", "n30_plga_150ns/1_xa_soln/N30_nopbcpp_1xa.xtc")

In [ ]:
n30_plga_ace

In [ ]:
len(n30_plga_ace.trajectory)

In [ ]:
poly_n30_ace = n30_plga_ace.select_atoms("resname sPLG PLG tPLG and not type H")

In [ ]:
n30_blks_ace = bavg_pers_cnt(5, poly_n30_ace, n30_plga_ace, lb_avg, 0 , 15000)

In [ ]:
n30_blks_ace["Avg persistence length"]

In [ ]:
n30_blks_ace["Avg Radius of gyration"]

In [ ]:
np.mean(n30_blks_ace["Avg persistence length"])

In [ ]:
np.std(n30_blks_ace["Avg persistence length"])